# Prediction gap test

Głównym celem tego notatnika jest przetestowanie naszego algorytmu liczącego prediction gap.

Robimy to dla zadania regresji, ponieważ dla zadania binarnej regresji logistycznej jest taki problem, że na koniec do wyników drzew przykładana jest funkcja sigmoid, co uniemożliwia szybkie analityczne obliczenie wyniku. Przy zadaniu regresji ten problem nie występuje.


## Konfiguracja

In [1]:
import os
while "notebooks" in os.getcwd():
    os.chdir("../")


In [29]:
from pathlib import Path
import pandas as pd

from src.decision_tree.tree import load_trees
from src.decision_tree.prediction_gap import (
    NormalPredictionGap,
    prediction_gap_on_single_feature_perturbation,
    prediction_gap_by_random_sampling,
    prediction_gap_by_exact_calc
)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
models_path = Path("models")
data_path = Path("data")
wine_model_name = "winequality_red"
wine_test_data_path = data_path / "wine_quality/test_winequality_red_scaled.csv"
housing_model_name = "housing"
housing_test_data_path = data_path / "housing_data/test_housing_scaled.csv"


In [4]:
stddev = 0.3


## Wczytanie danych i modelu

In [5]:
wine_trees = load_trees(models_path, wine_model_name)


In [6]:
wine_data = pd.read_csv(wine_test_data_path)
wine_data


,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,-0.413454,0.123905,-0.313113,-0.240375,-0.349975,-0.848716,-0.561586,-0.183745,-0.201591,-0.638220,-0.678644,5
1,1.310138,-0.937525,1.638205,-0.240375,1.371576,-0.944346,-0.865676,0.982285,-1.756618,2.312434,-0.960246,5
2,-1.160343,2.526090,-1.340122,-0.382271,-0.647527,-0.083669,-0.409542,-0.989366,1.871778,-1.169337,0.729364,6
3,-0.068735,0.598756,-0.877968,-0.311323,-0.307468,0.872638,0.411500,-0.194345,-0.136798,0.542042,0.447763,6
4,-1.217796,-0.490607,0.611196,-0.027532,-0.222453,-0.944346,-0.987312,-0.634256,1.288643,0.187963,0.541630,6
...,...,...,...,...,...,...,...,...,...,...,...,...
315,0.103624,-0.714066,0.662546,2.668484,-0.796303,-1.231239,-1.108948,-0.575955,-0.201591,-0.579207,1.480302,4
316,1.195232,0.626688,-0.159061,0.185312,0.372651,1.255161,0.198638,1.618302,-0.460762,0.069937,-0.490910,5
317,0.103624,-0.323013,-0.005010,-0.453218,-0.626274,0.203223,-0.257497,-0.830361,-0.979104,1.132173,0.635497,6
318,-0.585813,0.347364,-0.056360,-0.382271,-0.158692,0.107592,1.749495,-0.480552,-0.201591,-0.815259,-0.490910,5


In [7]:
housing_trees = load_trees(models_path, housing_model_name)


In [8]:
housing_data = pd.read_csv(housing_test_data_path)
housing_data


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,1.307575,-0.862335,-0.686477,-0.108529,0.019293,0.265378,0.111059,-1.045477,77700.0
1,-1.452618,0.987002,1.856182,-0.215333,-0.272609,-0.312139,-0.338825,0.137031,314300.0
2,-0.993418,1.581599,-0.527561,-0.341390,-0.469583,-0.355408,-0.532380,-0.600266,99100.0
3,-0.878618,1.370915,-0.924851,0.445661,0.342046,0.237120,0.435395,-0.294598,109400.0
4,-1.302879,0.982320,1.141059,-0.570583,-0.581123,-0.503761,-0.561151,-0.711385,76400.0
...,...,...,...,...,...,...,...,...,...
4123,-1.123192,0.809091,0.425936,-0.367518,-0.381775,0.021655,-0.398984,-0.079942,161500.0
4124,0.259400,-0.141327,-0.845393,4.616525,5.095534,4.930548,5.368429,-0.487884,87200.0
4125,0.638740,-0.768697,1.141059,-0.336348,-0.331939,-0.240611,-0.357134,-1.017579,112900.0
4126,0.598809,-0.675060,-0.765935,0.116539,0.671918,0.686594,0.537403,-0.632217,185100.0


In [9]:
sample_housing_data = housing_data.sample(n=200, random_state=42)
sample_housing_data


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
949,-0.065035,-0.567377,1.141059,-0.382186,0.045398,0.056977,0.100597,-0.629375,247900.0
3168,-0.883609,1.108731,0.267020,0.568967,0.353912,0.191202,0.385698,0.091447,129200.0
2080,-1.477574,1.075958,0.664310,-0.418857,-0.441105,-0.686553,-0.425140,0.133873,310300.0
1210,0.688653,-0.792107,1.299975,-0.350558,-0.396015,-0.097557,-0.307438,-0.265595,160800.0
2553,0.379192,-0.632923,-0.686477,0.477748,0.346792,0.403134,0.508632,0.152665,196800.0
...,...,...,...,...,...,...,...,...,...
3816,-0.119939,0.570316,-0.924851,-0.135574,-0.398388,-0.222067,-0.270819,0.189353,94400.0
1659,1.342514,-0.796789,0.664310,-0.522911,-0.258370,-0.494931,-0.412062,-1.395886,55000.0
3230,-1.168113,0.462633,1.856182,-0.648967,-0.642826,-0.827842,-0.613464,-0.065940,243800.0
4099,0.279366,0.205131,1.220517,-0.085151,0.088115,-0.084311,0.022129,-1.189809,50900.0


## Prediction gap dla każdego featura osobno

Widzimy że kolejność zwracana przez wersje `abs` oraz `squared` jest bardzo podobna, choć nie identyczna.

In [10]:
predgap = NormalPredictionGap(stddev)


In [11]:
%%time
single_abs_predgaps = prediction_gap_on_single_feature_perturbation(
    predgap, wine_trees, wine_data, squared=False)


Starting predgap calculation for density.
Starting predgap calculation for sulphates.
Starting predgap calculation for volatile_acidity.
Starting predgap calculation for residual_sugar.
Starting predgap calculation for free_sulfur_dioxide.
Starting predgap calculation for total_sulfur_dioxide.
Starting predgap calculation for fixed_acidity.
Starting predgap calculation for chlorides.
Starting predgap calculation for pH.
Starting predgap calculation for citric_acid.
Starting predgap calculation for alcohol.
CPU times: user 1min 26s, sys: 4.82 s, total: 1min 31s
Wall time: 1min 26s


In [12]:
single_abs_predgaps


,Feature,PredGap
10,alcohol,0.155754
1,sulphates,0.147723
7,chlorides,0.145904
2,volatile_acidity,0.144896
5,total_sulfur_dioxide,0.135442
3,residual_sugar,0.130128
9,citric_acid,0.114234
6,fixed_acidity,0.111571
4,free_sulfur_dioxide,0.107138
0,density,0.106114


In [13]:
%%time
single_sqr_predgaps = prediction_gap_on_single_feature_perturbation(
    predgap, wine_trees, wine_data, squared=True)


Starting predgap calculation for density.
Starting predgap calculation for sulphates.
Starting predgap calculation for volatile_acidity.
Starting predgap calculation for residual_sugar.
Starting predgap calculation for free_sulfur_dioxide.
Starting predgap calculation for total_sulfur_dioxide.
Starting predgap calculation for fixed_acidity.
Starting predgap calculation for chlorides.
Starting predgap calculation for pH.
Starting predgap calculation for citric_acid.
Starting predgap calculation for alcohol.
CPU times: user 1min 27s, sys: 4.78 s, total: 1min 31s
Wall time: 1min 25s


In [14]:
single_sqr_predgaps


,Feature,PredGap
10,alcohol,0.071638
1,sulphates,0.063286
2,volatile_acidity,0.062282
7,chlorides,0.061304
3,residual_sugar,0.061165
5,total_sulfur_dioxide,0.052848
4,free_sulfur_dioxide,0.041940
6,fixed_acidity,0.040056
9,citric_acid,0.040000
0,density,0.036612


In [15]:
two_best_features = set(single_sqr_predgaps["Feature"][:2])
all_but_two_worst_features = set(single_sqr_predgaps["Feature"][:-2])


## Prediction gap for two features

### Wnioski

Metoda losowa: Złożoność zależy liniowo od liczby iteracji, jak również liniowo od liczby datapointów. Wyniki nawet przy dużej liczbie iteracji widocznie odbiegają od wyników dokładnych (to znaczy nie są jakieś od czapy czy błędne, po prostu są niezbyt dokładne). Dla modelu winequality 1000 iteracji na 20 datapointach liczy się 1 minutę.

Metoda dokładna: Złożoność zależy liniowo od liczby datapointów. Poza tym, w praktyce złożoność zależy mocno od tego ile i jak ważnych featurów perturbujemy. Dla modelu winequality i 20 datapointów czasy obliczeń były: 17,5 minuty dla perturbacji dwóch najważniejszych featurów, 6,5 minuty dla perturbacji dwóch mniej ważnych featurów, 19 minut dla perturbacji trzech featurów. Natomiast jak spróbowałem zaburzać 9 featurów, to w godzinę nie dostałem wyniku nawet dla pierwszego datapointa. Warto przeanalizować algorytm i zobaczyć czy można go jakoś zoptymalizować.

In [40]:
import numpy as np

def rmse(y_pred, y_exact) -> float:
    return np.sqrt(((y_pred - y_exact) ** 2).mean())


In [28]:
print(stddev)


0.3


In [30]:
%%time
two_features_random_results = prediction_gap_by_random_sampling(
    wine_trees, wine_data, two_best_features, stddev=stddev, squared=True)


There are 320 datapoints in the dataset.
The following 2 features are subject to perturbation:
{'sulphates', 'alcohol'}
Starting prediction gap calculation by random sampling with 100 iterations.
CPU times: user 1min 42s, sys: 0 ns, total: 1min 42s
Wall time: 1min 42s


In [31]:
two_features_random_results[:20]


array([0.02799603, 0.12973671, 0.04107802, 0.03070649, 0.1093555 ,
       0.2920536 , 0.07141526, 0.09058493, 0.00999147, 0.0075794 ,
       0.05761052, 0.06389339, 0.03690714, 0.0816154 , 0.0042955 ,
       0.01368577, 0.51794217, 0.01740524, 0.0681178 , 0.02475463])

In [43]:
%%time
two_features_1000_iter_random_results = prediction_gap_by_random_sampling(
    wine_trees, wine_data[:20], two_best_features, stddev=stddev, squared=True, num_iter=1000)


There are 20 datapoints in the dataset.
The following 2 features are subject to perturbation:
{'sulphates', 'alcohol'}
Starting prediction gap calculation by random sampling with 1000 iterations.
CPU times: user 1min 2s, sys: 11.7 ms, total: 1min 2s
Wall time: 1min 2s


In [49]:
%%time
two_features_10000_iter_random_results = prediction_gap_by_random_sampling(
    wine_trees, wine_data[:20], two_best_features, stddev=stddev, squared=True, num_iter=10000)


There are 20 datapoints in the dataset.
The following 2 features are subject to perturbation:
{'sulphates', 'alcohol'}
Starting prediction gap calculation by random sampling with 10000 iterations.
CPU times: user 10min 31s, sys: 20.4 ms, total: 10min 31s
Wall time: 10min 32s


In [26]:
%%time
two_features_exact_result = prediction_gap_by_exact_calc(
    predgap, wine_trees, wine_data[:20], two_best_features, squared=True)


There are 20 datapoints in the dataset.
The following 2 features are subject to perturbation:
{'sulphates', 'alcohol'}
Starting exact prediction gap calculation.
Datapoint 0 returned predgap value of 0.03731612382401615.
Datapoint 1 returned predgap value of 0.1252209645299695.
Datapoint 2 returned predgap value of 0.05428991439731339.
Datapoint 3 returned predgap value of 0.029024615836689225.
Datapoint 4 returned predgap value of 0.11224871226358642.
Datapoint 5 returned predgap value of 0.2785850501329165.
Datapoint 6 returned predgap value of 0.09641113780559543.
Datapoint 7 returned predgap value of 0.07343150877137704.
Datapoint 8 returned predgap value of 0.026857657770105656.
Datapoint 9 returned predgap value of 0.010945918188328041.
Datapoint 10 returned predgap value of 0.07641482580332042.
Datapoint 11 returned predgap value of 0.03960350625307052.
Datapoint 12 returned predgap value of 0.029972484986642556.
Datapoint 13 returned predgap value of 0.10239222778234913.
Datapo

In [45]:
two_features_random_results[:20] - two_features_exact_result


array([-0.00932009,  0.00451574, -0.01321189,  0.00168187, -0.00289321,
        0.01346855, -0.02499588,  0.01715342, -0.01686618, -0.00336652,
       -0.01880431,  0.02428989,  0.00693466, -0.02077683,  0.00020629,
       -0.02230326,  0.14798567, -0.00495201, -0.04398865,  0.0025397 ])

In [46]:
rmse(two_features_random_results[:20], two_features_exact_result)


0.037064688434788745

In [47]:
two_features_1000_iter_random_results - two_features_exact_result


array([-0.00163512, -0.00734501, -0.01827149,  0.00132058, -0.00011846,
       -0.00976692, -0.02241525,  0.01472992, -0.00799429, -0.00094214,
       -0.00321946,  0.01223911, -0.00181794, -0.04108328,  0.00082562,
       -0.00674834,  0.09297517, -0.00094195, -0.04430147,  0.0034108 ])

In [48]:
rmse(two_features_1000_iter_random_results, two_features_exact_result)


0.026257263549314697

In [50]:
two_features_10000_iter_random_results - two_features_exact_result


array([ 0.00034398, -0.00491833, -0.01518421, -0.00166847, -0.0023242 ,
        0.00372941, -0.01392358,  0.01032379, -0.00676065, -0.00210408,
       -0.00164492,  0.01076239,  0.00047682, -0.03664943,  0.00120942,
       -0.00689738,  0.08465878, -0.00134585, -0.04099936,  0.00265963])

In [51]:
rmse(two_features_10000_iter_random_results, two_features_exact_result)


0.023447243126263256

In [63]:
np.mean(two_features_exact_result)


0.08297140178636664

In [64]:
np.mean(two_features_1000_iter_random_results)


0.08091640657841057

## Prediction gap for nine features

In [32]:
%%time
nine_features_random_results = prediction_gap_by_random_sampling(
    wine_trees, wine_data, all_but_two_worst_features, stddev=stddev, squared=True)


There are 320 datapoints in the dataset.
The following 9 features are subject to perturbation:
{'sulphates', 'volatile_acidity', 'residual_sugar', 'total_sulfur_dioxide', 'free_sulfur_dioxide', 'fixed_acidity', 'chlorides', 'citric_acid', 'alcohol'}
Starting prediction gap calculation by random sampling with 100 iterations.
CPU times: user 1min 40s, sys: 0 ns, total: 1min 40s
Wall time: 1min 40s


In [33]:
nine_features_random_results[:20]


array([0.15036154, 0.44905508, 0.13573757, 0.08494576, 0.12084828,
       0.25611358, 0.27412859, 0.20229642, 0.06082268, 0.05030419,
       0.0648766 , 0.13731049, 0.03389095, 0.18261715, 0.15353472,
       0.10820149, 0.3368857 , 0.04687097, 0.23106925, 0.16621358])

In [59]:
%%time
nine_features_1000_iter_random_results = prediction_gap_by_random_sampling(
    wine_trees, wine_data[:20], all_but_two_worst_features, stddev=stddev, squared=True, num_iter=1000)


There are 20 datapoints in the dataset.
The following 9 features are subject to perturbation:
{'sulphates', 'volatile_acidity', 'residual_sugar', 'total_sulfur_dioxide', 'free_sulfur_dioxide', 'fixed_acidity', 'chlorides', 'citric_acid', 'alcohol'}
Starting prediction gap calculation by random sampling with 1000 iterations.
CPU times: user 1min 4s, sys: 3.91 ms, total: 1min 4s
Wall time: 1min 4s


In [57]:
nine_features_1000_iter_random_results


array([0.11568296, 0.56259323, 0.14693399, 0.09526475, 0.14994631,
       0.27311876, 0.27180347, 0.1352542 , 0.04189183, 0.05286717,
       0.06751138, 0.15101628, 0.04591465, 0.16260566, 0.13961213,
       0.14080035, 0.36132008, 0.05099046, 0.23309117, 0.13668409])

In [65]:
%%time
more_features_exact_result = prediction_gap_by_exact_calc(
    predgap, wine_trees, wine_data[:20], {'volatile_acidity', 'residual_sugar', 'density'}, squared=True)


There are 20 datapoints in the dataset.
The following 3 features are subject to perturbation:
{'density', 'volatile_acidity', 'residual_sugar'}
Starting exact prediction gap calculation.
Datapoint 0 returned predgap value of 0.020679652235615097.
Datapoint 1 returned predgap value of 0.23023593568498377.
Datapoint 2 returned predgap value of 0.0557163047130985.
Datapoint 3 returned predgap value of 0.007825791323194081.
Datapoint 4 returned predgap value of 0.06161576436809076.
Datapoint 5 returned predgap value of 0.0758475194947762.
Datapoint 6 returned predgap value of 0.08665606672422425.
Datapoint 7 returned predgap value of 0.13226454452156822.
Datapoint 8 returned predgap value of 0.0176276183626842.
Datapoint 9 returned predgap value of 0.05086083380108456.
Datapoint 10 returned predgap value of 0.016721019191387423.
Datapoint 11 returned predgap value of 0.08148233476811524.
Datapoint 12 returned predgap value of 0.006987805505212817.
Datapoint 13 returned predgap value of 0.0

In [56]:
nine_features_random_results[:20] - nine_features_exact_result


NameError: name 'nine_features_exact_result' is not defined

In [ ]:
rmse(nine_features_random_results[:20], nine_features_exact_result)


In [ ]:
nine_features_1000_iter_random_results - nine_features_exact_result
